# Issue to Resolve

(((The EventLogger previously was being passed around as a global variable, which is not optimal. All classes should be subclasses of EventLogger I think, and all Owner objects should be subclasses of ERC721. Still not quite correct.))) <- Wrong



In [1]:
from datetime import datetime

In [2]:
# nftCollection = {}

class Eth:
    def __init__(self, amount):
        self.amount = amount
    
    def __add__(self, eth):
        self.amount += eth.amount
        return self
    
    def __sub__(self, eth):
        self.amount -= eth.amount
        return self
    
    def __neg__(self):
        return -self.amount
    
    def __pos__(self):
        return +self.amount
    
    def __repr__(self):
        return f"\N{greek capital letter xi}{self.amount}"
    

class Nft:
    def __init__(self, nftId, price, data, owner, contract):
        self.nftId = nftId
        self.price = price
        self.data = data
        self.owner = owner
        self.contract = contract
    
    @property
    def contractName(self):
        return self.contract._name
    
    @property
    def contractType(self):
        return type(self.contract).__name__
    
    def __repr__(self):
        return f"Nft({self.nftId}, {self.price}, {self.data}, {self.owner}, {self.contractName}, {self.contractType})"
    
    def __str__(self):
        return f"Nft({self.nftId})"
        
class Owner:
    def __init__(self, address, *args):
        self.address = '0X' + address # this is a string, not a hash
        nfts = set(filter(lambda x: type(x).__name__ == "Nft",args))
        ethSet = set(filter(lambda x: type(x).__name__ == "Eth",args))
        self.eth = Eth(0)
        for ethEl in ethSet:
            self.eth += ethEl
        self.isLender = False
        self.isOwed = 0.
        self.isBorrower = False
        self.owes = 0.
        self.eventLogger = EventLogger()
        if 'LISTOFALLCONTRACTS' not in globals():
            globals()['LISTOFALLCONTRACTS'] = []
        for nft in nfts:
            self.setInitialOwner(self.address,nft)
    
    @property
    def assets(self):
        return set(self.nfts).union({self.eth})
    
    @property
    def log(self):
        return self.eventLogger.log
    
    @property
    def nfts(self):
        nftStringsList = []
        for contractType in globals()['LISTOFALLCONTRACTS']:
            contracts = [k for k,v in globals().items() if type(v).__name__ == contractType]
            for contract in contracts:
                nftStringsList += [k for k,v in globals()[contract]._owners.items() if v == self.address]
        allNfts = [globals()[k] for k,v in globals().items() if type(v).__name__ == 'Nft']
        myNfts = [_ for _ in allNfts if _.nftId in nftStringsList]
        return myNfts
    
    def __add__(self, nft):
        currOwnerAddr = nft.owner
        if not currOwnerAddr:
            self.setInitialOwner(self.address,nft)
        else:
            self.approve(currOwnerAddr,nft)
            self.transferFrom(currOwnerAddr,self.address,nft)
    
    def __sub__(self, nft):
        self.burn(nft)
    
    def __contains__(self, asset):
        return asset in self.assets
    
    def __repr__(self):
        return f"Owner({self.address},{self.assets},{self.isLender},{self.isOwed},{self.isBorrower},{self.owes})"
    
    def __str__(self):
        return f"Owner({self.address})"
    
    def emit(self, *args):
        self.eventLogger.emit(*args)
    
    def approve(self, to, nft):
        #self.eventLogger.emit('Approval',str(self),ERC721Name)
        self.emit('Approval',str(self),nft.contractType,nft.contractName)
        theContract = self.getContract(nft)
        globals()[theContract].approve(to, nft.nftId, str(self))
    
    def transferFrom(self,addrFrom,to,nft):
        #self.eventLogger.emit('Transfer', addrFrom, to, nftId)
        self.emit('Transfer', addrFrom, to, nft.nftId)
        theContract = self.getContract(nft)
        globals()[theContract].transferFrom(addrFrom,to,nft.nftId,self.address)
    
    def getContract(self, nft):
        contractKeys = [k for k,v in dict(globals()).items() if type(v).__name__ == nft.contractType]
        assert len(contractKeys) > 0, "No ERC721 contracts have been created."
        theContract = list(filter(lambda x: globals()[x]._name == nft.contractName, contractKeys))[0]
        return theContract
    
    def setInitialOwner(self, to, nft):
        #self.eventLogger.emit('Transfer',to,nftId,ERC721Name)
        self.emit('SetInitialOwner',to,nft.nftId,nft.contractType,nft.contractName)
        theContract = self.getContract(nft)
        globals()[theContract].setInitialOwner(to,nft.nftId,str(self))
    
    def setInitialOwners(self, toList, nfts):
        #self.eventLogger.emit('Transfer',to,nftId,ERC721Name)
        nftIds = [_.nftId for _ in nfts]
        contractTypes = [_.contractType for _ in nfts]
        contractNames = [_.contractNames for _ in nfts]
        self.emit('SetInitialOwners',toList,nftIds,nftContracts,nftNames)
        theContracts = [self.getContract(_) for _ in contractNames]
        for theContract in list(set(theContracts)):
            contractIdxs = np.where(theContracts == theContract)
            currToList = np.array(toList)[contractIdxs]
            nftIds = np.array(nftIds)[contractIdxs]
            globals()[theContract].setInitialOwners(toList,nftIds,str(self))
    
    def burn(self,nft):
        self.emit('Burn',nft.nftId,nft.contractType,nft.contractName)
        theContract = self.getContract(nft)
        globals()[theContract]._burn(nft.nftId,str(self))
            
class EventLogger:
    def __init__(self):
        self.log = {}
        self.activeClasses = []
    
    def __iadd__(self, c):
        if str(c) not in self.activeClasses:
            self.activeClasses.append(str(c))
    
    @property
    def events(self):
        return list(self.log.keys())
    
    def emit(self,eventName,eventCaller,eventResponder,*args):
        #time, #type of event, #parameters
        currTime = datetime.now().strftime('%Y/%M/%d %H:%M:%S')
        result = [eventName, eventCaller, eventResponder, currTime, *args]
        if eventName not in self.log.keys():
            self.log[eventName] = [result]
        else:
            self.log[eventName] += [result]
    
    def __repr__(self):
        return f"EventLogger({self.log},{self.activeClasses})"

class ERC721:
    def __init__(self, name_, symbol_):
        self._name = name_
        self._symbol = symbol_
        self._owners = {}
        self._balances = {}
        self._tokenApprovals = {}
        self._operatorApprovals = {}
        self.eventLogger = EventLogger()
        self.allTokensAssigned = False
        self.nftsRemainingToAssign = 10000
        if 'LISTOFALLCONTRACTS' not in globals():
            globals()['LISTOFALLCONTRACTS'] = ['ERC721']
        elif 'ERC721' not in globals()['LISTOFALLCONTRACTS']:
            globals()['LISTOFALLCONTRACTS'] += ['ERC721']
    
    @property
    def log(self):
        return self.eventLogger.log
    
    def __repr__(self):
        return f"ERC721({self._name}, {self._symbol}, {self._owners}, {self._balances}, {self._tokenApprovals}, {self._operatorApprovals}, {self.eventLogger}, {self.allTokensAssigned})"
    
    def __str__(self):
        return f"ERC721({self._name})"
    
    def _exists(self, nftId):
        return nftId in self._owners.keys()
    
    def balanceOf(self, ownerAddress):
        if ownerAddress in self._balances.keys():
            return self._balances[ownerAddress]
        return None
    
    def ownerOf(self, nftId):
        if self._exists(nftId):
            return self._owners[nftId]
        return None
    
    def _approve(self, to, nftId):
        self._tokenApprovals[nftId] = to
        #self.eventLogger.emit("Approval",self.ownerOf(nftId), to, nftId)
        self.emit("Approval",self.ownerOf(nftId), to, nftId)
    
    def approve(self, to, nftId, msgSender):
        owner = self.ownerOf(nftId)
        assert owner != to, "ERC721: approve caller is not owner nor approved for all"
        #assert msgSender == owner | isApprovedForAll(owner, msgSender), "Error"
        self._approve(to, nftId)
    
    def isApprovedForAll(self,owner,operator):
        if owner in self._operatorApprovals.keys():
            return operator in self._operatorApprovals[owner].keys()
        return False
    
    def transferFrom(self,addrFrom,to,nftId,msgSender):
        assert self._isApprovedOrOwner(msgSender, nftId), "ERC721: transfer caller is not owner nor approved"
        self._transfer(addrFrom, to, nftId)
    
    def _isApprovedOrOwner(self, spender, nftId):
        assert self._exists(nftId), "ERC721: operator query for nonexistent token"
        owner = self.ownerOf(nftId)
        print(f'{spender=}, {nftId=}, {owner=}')
        print((spender == owner), (self.getApproved(nftId) == spender))
        return (spender == owner) | (self.getApproved(nftId) == spender) | (self.isApprovedForAll(owner, spender))
    
    def getApproved(self, nftId):
        assert self._exists(nftId), "ERC721: approved query for nonexistent token"
        return nftId in self._tokenApprovals.keys();
    
    def _transfer(self, addrFrom, to, nftId):
        assert self.ownerOf(nftId) == addrFrom, "ERC721: transfer from incorrect owner"
        assert to != None, "ERC721: transfer to the zero address"
        self._beforeTokenTransfer(addrFrom, to, nftId)
        self._approve(None, nftId)
        self._balances[addrFrom] -= 1
        self._balances[to] += 1
        self._owners[nftId] = to
        #self.eventLogger.emit("Transfer",addrFrom, to, nftId)
        self.emit("Transfer",addrFrom, to, nftId)
        self._afterTokenTransfer(addrFrom, to, nftId)
    
    def _afterTokenTransfer(self, addrFrom, to, nftId):
        pass
    
    def _safeMint(self, to, nftId, _data=""):
        self._mint(to, nftId)
        assert self._checkOnERC721Received(None, to, nftId, _data), "ERC721: transfer to non ERC721Receiver implementer"
    
    def _checkOnERC721Received(self, addrFrom, to, nftId, _data):
        #Add in later
        return True
    
    def _mint(self, to, nftId):
        assert to != None, "ERC721: mint to the zero address"
        assert not self._exists(nftId), "ERC721: token already minted"
        self._beforeTokenTransfer(None, to, nftId)
        if to not in self._balances.keys():
            self._balances[to] = 0
        self._balances[to] += 1
        self._owners[nftId] = to
        #self.eventLogger.emit("Transfer",None, to, nftId)
        self.emit("Transfer",None, to, nftId)
        self._afterTokenTransfer(None, to, nftId)
    
    def _beforeTokenTransfer(self, addrFrom, to, nftId):
        pass
    
    def _afterTokenTransfer(self, addrFrom, to, nftId):
        pass
    
    def setInitialOwner(self, to, nftId, msgSender):
        self.emit("Mint",msgSender,to,nftId)
        self._mint(to, nftId)
    
    def setInitialOwners(self, toList, nftIds, msgSender):
        [self.setInitialOwner(to, nftId, msgSender) for to,nftId in zip(toList,nftIds)]
    
    def emit(self,*args):
        self.eventLogger.emit(*args)
    
    def _burn(self, nftId, msgSender):
        owner = self.ownerOf(nftId)
        self._beforeTokenTransfer(owner, None, nftId)
        
        self._approve(None, nftId)
        
        self._balances[owner] -= 1
        del self._owners[nftId]
        
        self.emit("Transfer", msgSender, owner, None, nftId)
        
        self._afterTokenTransfer(owner, None, nftId)


# class N3RP:
#     def __init__(self, contractId, lender, lentAsset, beginPeriod, endPeriod):
#         if lentAsset not in lender:
#             raise ValueError(f"Lent asset {lentAsset} not in lender {lender}.")
#         self.contractId = contractId
#         self.lender = lender
#         self.lentAsset = lentAsset
#         self.beginPeriod = beginPeriod
#         self.endPeriod = endPeriod
#         self.borrower = None
#         self.collateralAsset = None
    
#     def __call__(self, borrower, collateralAsset):
#         if collateralAsset not in borrower:
#             raise ValueError(f"Collateral asset {collateralAsset} not in borrower {borrower}.")
        
#         #Is the collateral an nft? Or straight up eth.
        
#         self.lender.isLender = True
#         self.lender.isOwed = lentAsset.price
        
#         self.borrower = borrower
#         self.collateralAsset = collateralAssert
#         self.borrower.isBorrower = True
#         self.borrower.owes = self.lentAsset.price
        
        

In [37]:
class Address:
    def __init__(self, name):
        self.name = name

class N3RP:
    def __init__(self,
                 lenderAddr: Address,
                 borrowerAddr: Address,
                 initiatorAddr: Address,
                 nftAddr: Address,
                 basePayment: Eth,
                 collateral: Eth,
                 rentalDueDate: str, #Will be a ethereum blocktime, in reality
                 collateralPayoutPeriod: str, #Will be an integer number of blocks, in reality
                ):
        self.lender = lender
        self.borrower = borrower
        self.roleOfInitiator = roleOfInitiator
        self.nft = nft
        self.basePayment = basePayment
        self.collateral = collateral
        self.rentalDueDate = rentalDueDate
        self.collateralPayoutPeriod = collateralPayoutPeriod
        self.nftIsDeposited = False
        self.ethIsDeposited = False
    
    def depositNft(self, nftAddr: Address, msgSender: Address):
        assert self.lenderAddr == msgLender, f"Deposit Error: sender {msgSender} is not lender {self.lenderAddr}"
        assert self.nftAddr == nftAddr, f"Deposit Error: deposited nft {nftAddr} does not match the stipulated nft {self.nftAddr}"
        self.nftIsDeposited = True
    
    def depositEth(self, basePayment: Eth, collateral: Eth, depositorAddr: Address):
        pass
    

In [3]:
theERC721 = ERC721('CryptoPunks', 'CRYPTOPUNKS')

nft1 = Nft('JjCoin', 100, 'asdf123', '', theERC721)
nft2 = Nft('GgCoin', 1000, 'fdsa321', '', theERC721)
owner1 = Owner('Gg')
owner2 = Owner('Jj')
owner3 = Owner('xXxCryptoLordxXx')
# owner3.approve(owner2.address,nft2.nftId,'CryptoPunks')

In [6]:
owner3.setInitialOwner(owner1.address,nft1)

In [13]:
owner1.setInitialOwner(owner1.address,nft2)

In [15]:
owner1.nfts

[Nft(JjCoin, 100, asdf123, , CryptoPunks, ERC721),
 Nft(GgCoin, 1000, fdsa321, , CryptoPunks, ERC721)]

In [ ]:
owner2.nfts

In [ ]:
owner3.nfts

In [11]:
owner2.approve(owner2.address,nft1)

In [30]:
owner3.transferFrom(owner1.address,owner2.address,nft1)

AssertionError: ERC721: operator query for nonexistent token

In [19]:
owner1 - nft1

KeyError: None

In [20]:
owner1.nfts

[Nft(GgCoin, 1000, fdsa321, , CryptoPunks, ERC721)]

In [ ]:
theERC721._owners

In [22]:
owner2 + nft1

In [23]:
owner2.nfts

[Nft(JjCoin, 100, asdf123, , CryptoPunks, ERC721)]

In [ ]:
nft2

In [29]:
theERC721._owners

{'GgCoin': '0XGg'}

In [28]:
owner1 - nft1